In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812364 sha256=b47c355dc1db4b2f9c93b0d4d57100dddff54c854e13e026bf2797836c0ad11c
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [5]:
# import pyspark.sql classes and functions
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col, array_contains, isnan, when, count
from pyspark.sql.functions import lit, concat_ws, concat, collect_list, udf
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.appName("answers").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/28 02:45:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
path = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store"
df = spark.read.option("header",'True').option('delimiter', ',').csv(path, inferSchema = True)
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [8]:
path1 = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv"
df_nov = spark.read.option("header","True").option("delimiter",",").csv(path1, inferSchema = True)
#df_nov.show()

In [9]:
path2 = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv"
df_oct = spark.read.option("header","True").option("delimiter",",").csv(path2, inferSchema = True)
#df_oct.show()

In [10]:
df_nov

DataFrame[event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string]

# Question 1

In [12]:
df_cat_nov = df_nov.select(['category_code', 'event_type'])
df_cat_nov_view = df_cat_nov.filter(df_cat_nov["event_type"] == "view")
df_cat_nov_cart = df_cat_nov.filter(df_cat_nov["event_type"] == "cart")

In [17]:
df_cat_oct = df_oct.select(['category_code', 'event_type'])
df_cat_oct_view = df_cat_oct.filter(df_cat_oct["event_type"] == "view")
df_cat_oct_cart = df_cat_oct.filter(df_cat_oct["event_type"] == "cart")

In [13]:
df_cat_nov_view = df_cat_nov_view.na.drop(subset=["category_code"])

In [18]:
df_cat_oct_view = df_cat_oct_view.na.drop(subset=["category_code"])

In [14]:
df_cat_nov_cart = df_cat_nov_cart.na.drop(subset=["category_code"])

In [19]:
df_cat_oct_cart = df_cat_oct_cart.na.drop(subset=["category_code"])

In [11]:
import pyspark.sql.functions as f

In [15]:
df_cat_nov_view = df_cat_nov_view.groupBy('category_code').agg(f.count('event_type').alias("view_count"))
df_cat_nov_view

DataFrame[category_code: string, view_count: bigint]

In [20]:
df_cat_oct_view = df_cat_oct_view.groupBy('category_code').agg(f.count('event_type').alias("view_count"))
df_cat_oct_view

DataFrame[category_code: string, view_count: bigint]

In [16]:
df_cat_nov_cart = df_cat_nov_cart.groupBy('category_code').agg(f.count('event_type').alias("cart_count"))
df_cat_nov_cart

DataFrame[category_code: string, cart_count: bigint]

In [21]:
df_cat_oct_cart = df_cat_oct_cart.groupBy('category_code').agg(f.count('event_type').alias("cart_count"))
df_cat_oct_cart

DataFrame[category_code: string, cart_count: bigint]

In [23]:
df_cart_view_nov = df_cat_nov_cart.join(df_cat_nov_view, df_cat_nov_view.category_code == df_cat_nov_cart.category_code, "inner")
df_cart_view_nov

DataFrame[category_code: string, cart_count: bigint, category_code: string, view_count: bigint]

In [25]:
df_ratio_nov = df_cart_view_nov.withColumn("conversion_ratio", df_cart_view_nov["cart_count"]/df_cart_view_nov["view_count"])
df_ratio_nov = df_ratio_nov.sort("conversion_ratio", ascending=[False]) 
df_ratio_nov.show()

+--------------------+----------+--------------------+----------+--------------------+
|       category_code|cart_count|       category_code|view_count|    conversion_ratio|
+--------------------+----------+--------------------+----------+--------------------+
|   kids.fmcg.diapers|      2463|   kids.fmcg.diapers|     29296| 0.08407291097760787|
|appliances.kitche...|         8|appliances.kitche...|        99| 0.08080808080808081|
|electronics.audio...|    131133|electronics.audio...|   1644910| 0.07972047102881008|
|electronics.smart...|   1159966|electronics.smart...|  14832387| 0.07820494435588823|
|appliances.enviro...|     11914|appliances.enviro...|    194075| 0.06138863841298467|
|medicine.tools.to...|      1274|medicine.tools.to...|     20764| 0.06135619341167405|
|appliances.kitche...|     15938|appliances.kitche...|    285186|0.055886333831253986|
|appliances.person...|      4855|appliances.person...|     88564| 0.05481911386116255|
| stationery.cartrige|       644| stationer

In [26]:
df_cart_view_oct = df_cat_oct_cart.join(df_cat_oct_view, df_cat_oct_view.category_code == df_cat_oct_cart.category_code, "inner")
df_cart_view_oct

DataFrame[category_code: string, cart_count: bigint, category_code: string, view_count: bigint]

In [27]:
df_ratio_oct = df_cart_view_oct.withColumn("conversion_ratio", df_cart_view_oct["cart_count"]/df_cart_view_oct["view_count"])
df_ratio_oct = df_ratio_oct.sort("conversion_ratio", ascending=[False]) 
df_ratio_oct.show()

+--------------------+----------+--------------------+----------+--------------------+
|       category_code|cart_count|       category_code|view_count|    conversion_ratio|
+--------------------+----------+--------------------+----------+--------------------+
|electronics.smart...|    549765|electronics.smart...|  10619448| 0.05176964000388721|
|electronics.audio...|     51143|electronics.audio...|   1018542|0.050211969658590414|
|medicine.tools.to...|       522|medicine.tools.to...|     13974|  0.0373550880206097|
|electronics.video.tv|     36224|electronics.video.tv|   1055961|0.034304297223098204|
|  electronics.tablet|      9140|  electronics.tablet|    301992|0.030265702402712653|
|appliances.kitche...|      4717|appliances.kitche...|    164954|0.028595850964511317|
|appliances.enviro...|      3916|appliances.enviro...|    138784|0.028216509107678117|
|appliances.kitche...|     21977|appliances.kitche...|    831279| 0.02643757390719602|
|     appliances.iron|      4095|     appli

## its clearly visible they are not same for both months

# Question 2

In [53]:
df_nov = df_nov.na.drop(subset = ["category_code"])
df_oct = df_oct.na.drop(subset = ["category_code"])

In [54]:
df_nov_rev = df_nov.select(["category_code", "price", "event_type"])
df_oct_rev = df_oct.select(["category_code", "price", "event_type"])

In [55]:
df_nov_rev = df_nov_rev.filter(df_nov_rev["event_type"] == "purchase")
df_oct_rev = df_oct_rev.filter(df_oct_rev["event_type"] == "purchase")

In [56]:
df_nov_rev = df_nov_rev.groupBy("category_code").agg(f.sum("price").alias("total_revenue_nov"))
df_oct_rev = df_oct_rev.groupBy("category_code").agg(f.sum("price").alias("total_revenue_oct"))

In [57]:
df_oct_rev = df_oct_rev.withColumnRenamed("category_code", "category_code_oct")
df_nov_rev = df_nov_rev.withColumnRenamed("category_code", "category_code_nov")
df_oct_rev

DataFrame[category_code_oct: string, total_revenue_oct: double]

In [58]:
df_oct_nov_rev = df_oct_rev.join(df_nov_rev, df_oct_rev["category_code_oct"] == df_nov_rev["category_code_nov"])
df_oct_nov_rev

DataFrame[category_code_oct: string, total_revenue_oct: double, category_code_nov: string, total_revenue_nov: double]

In [59]:
df_rev_diff = df_oct_nov_rev.withColumn("difference", df_oct_nov_rev["total_revenue_nov"] - df_oct_nov_rev["total_revenue_oct"])
df_rev_diff

DataFrame[category_code_oct: string, total_revenue_oct: double, category_code_nov: string, total_revenue_nov: double, difference: double]

In [60]:
df_rev_diff.show(10)

+--------------------+------------------+--------------------+------------------+------------------+
|   category_code_oct| total_revenue_oct|   category_code_nov| total_revenue_nov|        difference|
+--------------------+------------------+--------------------+------------------+------------------+
|    computers.ebooks|47831.619999999995|    computers.ebooks|64230.219999999994|           16398.6|
|computers.periphe...|26891.610000000004|computers.periphe...|28976.320000000007|2084.7100000000028|
|appliances.kitche...|           3683.97|appliances.kitche...| 4721.330000000001| 1037.360000000001|
|electronics.video...| 37059.95000000001|electronics.video...|          55364.94| 18304.98999999999|
|electronics.camer...|140711.99000000002|electronics.camer...|         157889.89|17177.899999999994|
|       apparel.shirt|1940.5699999999997|       apparel.shirt| 6849.440000000002| 4908.870000000003|
|electronics.audio...|3539126.9900000007|electronics.audio...| 5669502.490000021|  2130375.

In [61]:
df_rev_diff_increasing = df_rev_diff.filter(df_rev_diff["difference"] > 0)
df_rev_diff_increasing

DataFrame[category_code_oct: string, total_revenue_oct: double, category_code_nov: string, total_revenue_nov: double, difference: double]

In [62]:
df_rev_diff_decreasing = df_rev_diff.filter(df_rev_diff["difference"] < 0)
df_rev_diff_decreasing

DataFrame[category_code_oct: string, total_revenue_oct: double, category_code_nov: string, total_revenue_nov: double, difference: double]

## Top 10 Increasing revenue codes

In [63]:
df_rev_diff_increasing = df_rev_diff_increasing.sort("difference", ascending = [False])
df_rev_diff_increasing.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|   category_code_oct|   total_revenue_oct|   category_code_nov|   total_revenue_nov|          difference|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|electronics.smart...|1.5704962336999914E8|electronics.smart...|1.7782166160999876E8|2.0772038239999622E7|
|electronics.video.tv|   8423407.859999994|electronics.video.tv|1.2457151160000006E7|   4033743.300000012|
|electronics.audio...|  3539126.9900000007|electronics.audio...|   5669502.490000021|    2130375.50000002|
|  electronics.clocks|   4818305.469999999|  electronics.clocks|   6552737.249999998|  1734431.7799999993|
|  computers.notebook|          8979887.25|  computers.notebook|1.0678429710000006E7|  1698542.4600000065|
|appliances.kitche...|   4658646.590000002|appliances.kitche...|          5801906.33|  1143259.7399999984|
|appliances.enviro...|  1716425.40999

## Top 10 decreasing revenue codes

In [64]:
df_rev_diff_decreasing = df_rev_diff_decreasing.sort("difference", ascending = [True])
df_rev_diff_decreasing.show(10)

+--------------------+------------------+--------------------+------------------+-------------------+
|   category_code_oct| total_revenue_oct|   category_code_nov| total_revenue_nov|         difference|
+--------------------+------------------+--------------------+------------------+-------------------+
|  electronics.tablet|1610973.7799999998|  electronics.tablet|1520253.0100000005| -90720.76999999932|
|         kids.skates|202582.28999999995|         kids.skates|         118786.27| -83796.01999999995|
|       sport.bicycle|203694.72999999995|       sport.bicycle|158871.60999999993|-44823.120000000024|
|auto.accessories....| 523241.7999999999|auto.accessories....| 487207.6299999999|-36034.169999999984|
|  apparel.shoes.keds|          188218.5|  apparel.shoes.keds|169055.61000000002|-19162.889999999985|
|construction.tool...|         173856.12|construction.tool...|166201.41999999998| -7654.700000000012|
|construction.tool...|          34624.83|construction.tool...|          28959.86| 

# Question 3

In [65]:
df_nov_new = spark.read.option("header","True").option("delimiter",",").csv(path1, inferSchema = True)
df_oct_new = spark.read.option("header","True").option("delimiter",",").csv(path2, inferSchema = True)


In [84]:
df_brand_nov = df_nov_new.select(["brand", "event_type"])
df_brand_oct = df_oct_new.select(["brand", "event_type"])

In [85]:
df_brand_nov = df_brand_nov.na.drop(subset = ["brand"])
df_brand_oct = df_brand_oct.na.drop(subset = ["brand"])

In [86]:
df_brand_view_nov = df_brand_nov.filter(df_brand_nov["event_type"] == "view")
df_brand_cart_nov = df_brand_nov.filter(df_brand_nov["event_type"] == "cart")

In [87]:
df_brand_view_oct = df_brand_oct.filter(df_brand_oct["event_type"] == "view")
df_brand_cart_oct = df_brand_oct.filter(df_brand_oct["event_type"] == "cart")

In [88]:
df_brand_nov_view = df_brand_view_nov.groupBy('brand').agg(f.count('event_type').alias("view_count"))
df_brand_nov_view

DataFrame[brand: string, view_count: bigint]

In [89]:
df_brand_nov_cart = df_brand_cart_nov.groupBy('brand').agg(f.count('event_type').alias("cart_count"))
df_brand_nov_cart

DataFrame[brand: string, cart_count: bigint]

In [90]:
df_brand_oct_view = df_brand_view_oct.groupBy('brand').agg(f.count('event_type').alias("view_count"))
df_brand_oct_view

DataFrame[brand: string, view_count: bigint]

In [91]:
df_brand_oct_cart = df_brand_cart_oct.groupBy('brand').agg(f.count('event_type').alias("cart_count"))
df_brand_oct_cart

DataFrame[brand: string, cart_count: bigint]

In [134]:
df_brand_nov_view = df_brand_nov_view.withColumnRenamed("brand","brand_nov_view")
df_brand_oct_view = df_brand_oct_view.withColumnRenamed("brand","brand_oct_view")

In [135]:
df_brand_nov_cart = df_brand_nov_cart.withColumnRenamed("brand","brand_nov_cart")
df_brand_oct_cart = df_brand_oct_cart.withColumnRenamed("brand","brand_oct_cart")

In [136]:
df_view_cart_nov = df_brand_nov_view.join(df_brand_nov_cart, df_brand_nov_view["brand_nov_view"] == df_brand_nov_cart["brand_nov_cart"])
df_view_cart_oct = df_brand_oct_view.join(df_brand_oct_cart, df_brand_oct_view["brand_oct_view"] == df_brand_oct_cart["brand_oct_cart"])
df_view_cart_nov

DataFrame[brand_nov_view: string, view_count: bigint, brand_nov_cart: string, cart_count: bigint]

In [137]:
df_view_cart_oct

DataFrame[brand_oct_view: string, view_count: bigint, brand_oct_cart: string, cart_count: bigint]

In [138]:
df_view_cart_nov = df_view_cart_nov.withColumn("Conversion_Ratio", df_view_cart_nov["cart_count"]/df_view_cart_nov["view_count"])
df_view_cart_oct = df_view_cart_oct.withColumn("Conversion_Ratio", df_view_cart_oct["cart_count"]/df_view_cart_oct["view_count"])

In [139]:
df_view_cart_nov = df_view_cart_nov.sort("Conversion_Ratio", ascending = [False])
df_view_cart_nov.show(10)

+--------------+----------+--------------+----------+-------------------+
|brand_nov_view|view_count|brand_nov_cart|cart_count|   Conversion_Ratio|
+--------------+----------+--------------+----------+-------------------+
|   pixiebelles|         5|   pixiebelles|         2|                0.4|
|      autolite|         5|      autolite|         2|                0.4|
|         ivaru|       177|         ivaru|        51|  0.288135593220339|
|     bigklever|         4|     bigklever|         1|               0.25|
|         lamel|         9|         lamel|         2| 0.2222222222222222|
|     superfine|        19|     superfine|         4|0.21052631578947367|
|  merejbaspasy|        91|  merejbaspasy|        19| 0.2087912087912088|
|           htl|         5|           htl|         1|                0.2|
|          basf|       126|          basf|        25| 0.1984126984126984|
|      yoshioki|        54|      yoshioki|        10|0.18518518518518517|
+--------------+----------+-----------

In [140]:
df_view_cart_oct = df_view_cart_oct.sort("Conversion_Ratio", ascending = [False])
df_view_cart_oct.show(10)

+--------------------+----------+--------------------+----------+--------------------+
|      brand_oct_view|view_count|      brand_oct_cart|cart_count|    Conversion_Ratio|
+--------------------+----------+--------------------+----------+--------------------+
|                  ea|     13025|                  ea|      1855| 0.14241842610364683|
|             diamond|       594|             diamond|        44| 0.07407407407407407|
|             samsung|   4806630|             samsung|    303249| 0.06308973230725061|
|         homeelement|       101|         homeelement|         6|  0.0594059405940594|
|               steba|        85|               steba|         5|0.058823529411764705|
|              domyos|      2178|              domyos|       128| 0.05876951331496786|
|               goo.n|       933|               goo.n|        53| 0.05680600214362272|
|sonyinteractiveen...|       985|sonyinteractiveen...|        55| 0.05583756345177665|
|               apple|   3770597|          

# Question 4

In [97]:
df_rev_brand_nov = df_nov_new.select(["brand", "price", "event_type"])
df_rev_brand_oct = df_oct_new.select(["brand", "price", "event_type"])

In [98]:
df_rev_brand_nov = df_rev_brand_nov.na.drop(subset = ["brand"])
df_rev_brand_oct = df_rev_brand_oct.na.drop(subset = ["brand"])

In [99]:
df_rev_nov = df_rev_brand_nov.filter(df_rev_brand_nov["event_type"] == "purchase")
df_rev_oct = df_rev_brand_oct.filter(df_rev_brand_oct["event_type"] == "purchase")

In [100]:
df_rev_nov = df_rev_nov.groupBy("brand").agg(f.sum("price").alias("total_revenue_nov"))
df_rev_oct = df_rev_oct.groupBy("brand").agg(f.sum("price").alias("total_revenue_oct"))

In [130]:
df_rev_nov = df_rev_nov.withColumnRenamed("brand", "brand_nov")
df_rev_oct = df_rev_oct.withColumnRenamed("brand", "brand_oct")

In [131]:
df_nov_oct = df_rev_nov.join(df_rev_oct, df_rev_nov["brand_nov"] == df_rev_oct["brand_oct"],"inner")

## Top 10 brands for increasing revenues

In [132]:
df_nov_oct = df_nov_oct.withColumn("difference", df_nov_oct["total_revenue_nov"] - df_nov_oct["total_revenue_oct"])
df_nov_oct = df_nov_oct.sort("difference", ascending = [False])
df_nov_oct.show(10) #increasing

+---------+--------------------+---------+-------------------+--------------------+
|brand_nov|   total_revenue_nov|brand_oct|  total_revenue_oct|          difference|
+---------+--------------------+---------+-------------------+--------------------+
|    apple|1.2751252488000059E8|    apple|1.112092688200001E8|1.6303256060000494E7|
|  samsung| 5.486988086999983E7|  samsung|4.640753261000007E7|   8462348.259999767|
|   xiaomi|1.1259865960000008E7|   xiaomi|  9194033.289999995|   2065832.670000013|
|       lg|          5239018.76|       lg| 3387887.9600000004|  1851130.7999999993|
|     sony|   3862886.300000004|     sony| 2478196.6799999997|  1384689.6200000043|
|     oppo|  3488540.7599999965|     oppo| 2412959.7600000002|  1075580.9999999963|
|   lenovo|   2698106.299999999|   lenovo| 1752638.5299999993|   945467.7699999996|
|     asus|  1665811.5900000003|     asus|          970019.39|   695792.2000000003|
|    artel|   1642215.040000001|    artel| 1034152.4099999999|    608062.630

## Top 10 brands for decreasing revenues

In [133]:
df_nov_oct_decr = df_nov_oct.sort("difference", ascending = [True])
df_nov_oct_decr.show(10)

+---------+------------------+---------+------------------+-------------------+
|brand_nov| total_revenue_nov|brand_oct| total_revenue_oct|         difference|
+---------+------------------+---------+------------------+-------------------+
|     acer| 3347306.530000001|     acer|3576719.5200000014|-229412.99000000022|
|     peda|48278.939999999995|     peda|219342.36999999997|-171063.42999999996|
| omabelle|201897.13000000003| omabelle| 347730.8700000001|-145833.74000000008|
|   hubert| 98882.01000000005|   hubert|219706.36000000002|-120824.34999999996|
|   huawei|        4780682.35|   huawei| 4883421.739999999|-102739.38999999966|
| elenberg|160643.44000000006| elenberg|259771.00000000012| -99127.56000000006|
|     skad|         115344.12|     skad|189851.01999999996| -74506.89999999997|
|    force|222173.14999999994|    force|286056.18999999994| -63883.04000000001|
|     beko| 904347.0499999997|     beko| 963940.5899999999|-59593.540000000154|
|    genau|29730.070000000007|    genau|

# Question 5 and Question 6

In [118]:
df_time_nov = df_nov_new.select(["brand","event_type","price","event_time"])
df_time_oct = df_oct_new.select(["brand","event_type","price","event_time"])

In [119]:
df_time_nov = df_time_nov.na.drop(subset = ["brand"])
df_time_oct = df_time_oct.na.drop(subset = ["brand"])

In [120]:
df_time_nov = df_time_nov.withColumn("hour", f.hour("event_time"))
df_time_nov = df_time_nov.withColumn("minute", f.minute("event_time"))
df_time_nov = df_time_nov.withColumn("second", f.second("event_time"))

In [122]:
df_time_oct = df_time_oct.withColumn("hour", f.hour("event_time"))
df_time_oct = df_time_oct.withColumn("minute", f.minute("event_time"))
df_time_oct = df_time_oct.withColumn("second", f.second("event_time"))

In [123]:
df_time_nov = df_time_nov.filter(((df_time_nov["hour"] >= 0 ) & (df_time_nov["hour"] <= 9)) | ((df_time_nov["hour"] == 10) & (df_time_nov["minute"] == 0) & (df_time_nov["second"] == 0)))
df_time_oct = df_time_oct.filter(((df_time_oct["hour"] >= 0 ) & (df_time_oct["hour"] <= 9)) | ((df_time_oct["hour"] == 10) & (df_time_oct["minute"] == 0) & (df_time_oct["second"] == 0)))

In [124]:
df_time_nov = df_time_nov.filter(df_time_nov["event_type"] == "purchase")
df_time_oct = df_time_oct.filter(df_time_oct["event_type"] == "purchase")

In [125]:
df_time_nov = df_time_nov.groupBy("brand").agg(f.sum("price"))

In [126]:
df_time_nov = df_time_nov.withColumnRenamed("sum(price)", " revenue_nov")

In [127]:
df_time_oct = df_time_oct.groupBy("brand").agg(f.sum("price").alias("revenue_oct"))

In [128]:
df_nov_rev_desc = df_time_nov.sort(" revenue_nov",ascending = [False])
df_nov_rev_desc.show()

+--------+--------------------+
|   brand|         revenue_nov|
+--------+--------------------+
|   apple| 5.285540107000014E7|
| samsung|2.4343482049999952E7|
|  xiaomi|          4728145.15|
|      lg|          2220124.87|
|  huawei|  1965448.4700000002|
| lucente|          1542212.88|
|    oppo|  1509522.4999999981|
|    sony|  1506703.0700000003|
|    acer|  1375710.4299999997|
|  lenovo|          1073758.32|
|   artel|           751945.93|
|    asus|           708319.69|
|   bosch|           701336.12|
| indesit|   562303.6399999999|
|      hp|   556653.0499999999|
|   haier|   474071.9899999999|
|cordiant|   418524.2600000001|
|    beko|   397114.0899999999|
|dauscher|   365249.8300000001|
|   midea|   262774.1099999999|
+--------+--------------------+
only showing top 20 rows



In [129]:
df_oct_rev_desc = df_time_oct.sort("revenue_oct",ascending = [False])
df_oct_rev_desc.show()

+--------+--------------------+
|   brand|         revenue_oct|
+--------+--------------------+
|   apple|5.0918605740000114E7|
| samsung|2.2968069980000094E7|
|  xiaomi|   4424108.319999999|
|  huawei|  2288629.6799999997|
|      lg|  1647661.0500000005|
|    acer|  1645916.2900000003|
| lucente|  1543609.4399999997|
|    oppo|          1167686.79|
|    sony|  1135915.6100000003|
|  lenovo|   798221.2800000004|
| indesit|           647301.35|
|   bosch|   597665.8200000001|
|      hp|   594798.2499999999|
|   artel|   537671.8700000002|
|    beko|   473414.1400000001|
|    asus|           437766.09|
|   haier|            404939.0|
|dauscher|           313623.22|
|cordiant|  277413.07000000007|
| philips|  242311.11000000002|
+--------+--------------------+
only showing top 20 rows



## We promote the top brands, i.e. the brands that generate more revenue. Thus, sorted the brands in descending order of their revenue

# Question 7

## No the brands for both Question 3 and 4 are not the same as we can see from the tables obtained in the results of question 3 and 4